Query 1
“Modality” ECOG #todo
“Gender” Male
“Primary Language” Afrikaans
“analysis_step” contains “preprocessing”
“Musical Training” yes
“scan” FingerLoc

work around modality for now
Query 1 output parameters required

project
subject
session
expected: demo2, PD005, ECOG

In [1]:
import pandas as pd
import setup
import nexussdk as nexus
from SPARQLWrapper import SPARQLWrapper, JSON, POST, GET, POSTDIRECTLY, CSV
from questionnaire_query_helper import generate_sparql_compliant_variable_name, generate_question_hash, query_constructor
config = setup.config
import NexusSparqlQuery as qns

# get script arguments
nexus_deployment = config['nexus']['deployment']
org = config['nexus']['org']
project = config['nexus']['project']
token = config['nexus']['token_file']
f = open(token, 'r')
token = f.read().rstrip()

sparqlview_endpoint = setup.sparqlview_endpoint
sparqlview_wrapper = setup.sparqlview_wrapper

nexus = setup.nexus

print("Connected")


Connected


In [2]:
prefix = """
prefix nxv: <https://bluebrain.github.io/nexus/vocabulary/>
PREFIX fhir: <http://hl7.org/fhir/> 
PREFIX kcnietl: <http://www.camh.ca/kcnietl/> 
PREFIX nidm: <http://purl.org/nidash/nidm#> 
PREFIX prov: <http://www.w3.org/ns/prov#> 
PREFIX sio: <http://semanticscience.org/resource/> 
PREFIX dct: <http://purl.org/dc/terms/> 
"""

select_list = [
    '?study_id',
    '?subject_id',
    '?session_id',
    '?acquisition_id',
    '?acquisition_description'
]

base_query = """
        ##### Get Study Information
        ?study_uri a fhir:ResearchStudy .
        ?study_uri fhir:ResearchStudy.identifier/fhir:Identifier.value/fhir:value ?study_id .
        
        #### Get Subject Information        
        ?subject_uri fhir:ResearchSubject.study/fhir:link ?study_uri .
        ?subject_uri fhir:ResearchSubject.identifier/fhir:Identifier.value/fhir:value ?subject_id .
        
        #### Get Session Information
        ?session_uri a nidm:Session .
        ?session_uri prov:wasAssociatedWith ?subject_uri .
        ?session_uri dct:identifier ?session_id .
        
        #### Get Acquisition Information
        ?acquisition a nidm:Acquisition .
        ?acquisition dct:isPartOf ?session_uri .
        ?acquisition dct:identifier ?acquisition_id .
        ?acquisition dct:description ?acquisition_description .
"""

subject_question_list = ['Gender', 'Primary Language', 'Musical Training']
for question in subject_question_list:
    sparql_compliant_variable_name = generate_sparql_compliant_variable_name(question)
    select_list.append(sparql_compliant_variable_name)
    base_query = base_query + query_constructor('?subject_uri', question, sparql_compliant_variable_name)

session_question_list = ['Analysis Steps']
for question in session_question_list:
    sparql_compliant_variable_name = generate_sparql_compliant_variable_name(question)
    select_list.append(sparql_compliant_variable_name)
    base_query = base_query + query_constructor('?session_uri', question, sparql_compliant_variable_name)
    

In [3]:
query = f"""
{prefix}

select distinct
    {' '.join(select_list)}
where
    {{
        {base_query}
    }}
"""

In [4]:
print(query)



prefix nxv: <https://bluebrain.github.io/nexus/vocabulary/>
PREFIX fhir: <http://hl7.org/fhir/> 
PREFIX kcnietl: <http://www.camh.ca/kcnietl/> 
PREFIX nidm: <http://purl.org/nidash/nidm#> 
PREFIX prov: <http://www.w3.org/ns/prov#> 
PREFIX sio: <http://semanticscience.org/resource/> 
PREFIX dct: <http://purl.org/dc/terms/> 


select distinct
    ?study_id ?subject_id ?session_id ?acquisition_id ?acquisition_description ?Gender ?Primary___Language ?Musical___Training ?Analysis___Steps
where
    {
        
        ##### Get Study Information
        ?study_uri a fhir:ResearchStudy .
        ?study_uri fhir:ResearchStudy.identifier/fhir:Identifier.value/fhir:value ?study_id .
        
        #### Get Subject Information        
        ?subject_uri fhir:ResearchSubject.study/fhir:link ?study_uri .
        ?subject_uri fhir:ResearchSubject.identifier/fhir:Identifier.value/fhir:value ?subject_id .
        
        #### Get Session Information
        ?session_uri a nidm:Session .
        

In [5]:
df = qns.sparql2dataframe(qns.query_sparql(query, sparqlview_wrapper))
df

,study_id,subject_id,session_id,acquisition_id,acquisition_description,Gender,Primary___Language,Musical___Training,Analysis___Steps
0,nexus2,NB002,NB002_ECOG_1,DurR1,EDF,male,Afrikaans,yes,"preprocessing, filtering, epoching"
1,nexus2,NB002,NB002_ECOG_1,DurR5,EDF,male,Afrikaans,yes,"preprocessing, filtering, epoching"
2,nexus2,NB002,NB002_ECOG_1,DurR4,EDF,male,Afrikaans,yes,"preprocessing, filtering, epoching"
3,nexus2,NB002,NB002_ECOG_1,DurR3,EDF,male,Afrikaans,yes,"preprocessing, filtering, epoching"
4,nexus2,NB002,NB002_ECOG_1,DurR2,EDF,male,Afrikaans,yes,"preprocessing, filtering, epoching"
...,...,...,...,...,...,...,...,...,...
508,nexus1,NA001,NA001_MR_1,2,SD123_MR_V1_anat,male,Afrikaans,no,None
509,nexus1,NA001,NA001_MR_1,2,SD123_MR_V1_anat,female,Afrikaans,no,None
510,nexus1,NA001,NA001_MR_1,2,SD123_MR_V1_anat,male,Chinese,no,None
511,nexus1,NA001,NA001_MR_1,2,SD123_MR_V1_anat,female,Chinese,no,None


In [6]:
df.loc[
    (df['acquisition_id'] == 'FingerLoc')
    & (df['Gender'] == 'male')
    & (df['Primary___Language'] == 'Afrikaans')
    & (df['Analysis___Steps'].str.find('preprocessing')!=-1)
    & (df['Musical___Training'] == 'yes')
    ].drop_duplicates()

,study_id,subject_id,session_id,acquisition_id,acquisition_description,Gender,Primary___Language,Musical___Training,Analysis___Steps
5,nexus2,NB002,NB002_ECOG_1,FingerLoc,EDF,male,Afrikaans,yes,"preprocessing, filtering, epoching"


End